# Kafi - Your Swiss Army Knife for Kafka Scripting
### Ralph Debusmann, Migros-Genossenschafts-Bund, Zürich, Switzerland
![alt text](migros.png "Migros")


***

Welcome to a typical Kafka administrator nightmare...
![alt text](nightmare.jpg "Nightmare")



...and how Kafi, your Swiss army knife for Kafka, will help us to get out of it.

Kafi is an Open Source Python library for powerful + convenient Kafka scripting.

***

The situation: We have a JSONSchema-serialized topic "products" and a qualification test is looming for an application consuming from that topic.

***

Let's run the qualification test...

(run consumer.py)

***

What's wrong with the topic?

In [16]:
from kafi.kafi import *

# Connect to Kafka
c = Cluster("local")

# Read the first message of the topic
c.head("products", type="bytes", n=1)

[{'topic': 'products',
  'headers': None,
  'partition': 0,
  'offset': 0,
  'timestamp': (1, 1731606949702),
  'key': None,
  'value': b'{"name": "Semi-hard cheese made from pasteurised milk", "url": "https://www.migros.ch/en/product/212430300000", "id": "212430300000", "price_unit": "2.16/100g", "weight": "12 x 22g", "pryce": "5.70"}'}]

***

How many message values do not start with the magic byte (0)?

In [17]:
x = c.filter("products", type="bytes", filter_function=lambda x: x["value"][0] != 0)
print(len(x[0]))

Read: 1000
Read: 2000
100


***

Let's do a backup of these faulty messages to a topic backed by Kafi's Kafka emulator (editable clone of the copy on your local disk).

In [18]:
# Connect to Kafi's Kafka emulator
l = Local("local")

# (Re)create the backup topic
l.retouch("products_backup")

# Copy the first 100 messages from the Kafka topic "products" to the topic "products_backup" on the Kafka emulator
c.cp("products", l, "products_backup", source_type="json", target_type="json", n=100)

(100, 100)

***

And then delete the first 100 messages on the real Kafka topic.

In [19]:
c.delete_records({"products": {0: 100}})

In [20]:
c.watermarks("products")

{'products': {0: (100, 2295)}}

***

Let's run the qualification test again...

(run consumer.py)

***

However, we do have to bring back the first 100 messages (the producers are not available).

In [21]:
# Get the schema ID of the first good message value
z = c.head("products", type="bytes", n=1)
sid = int.from_bytes(z[0]["value"][1:5], "big")

# Try to copy the backup to Kafka (this time - correctly JSONSchema-serialized)
l.cp("products_backup", c, "products", target_value_type="jsonschema", target_value_schema_id=sid)

SerializationError: 'price' is a required property

***

SerializationError: 'price' is a required property...

Let's check in Excel...

In [22]:
# Copy the backup to an Excel file
l.to_file("products_backup", l, "products_backup.xlsx", n=100)

100

(show products_backup.xlsx in Excel)

***

Because the topic is on Kafi's Kafka emulator and thus editable, we can just fix the messages in-place.

(open the editable partition file and fix)

***

And let's try to bring back the messages to the Kafka topic once again...

In [23]:
l.cp("products_backup", c, "products", target_value_type="jsonschema", target_value_schema_id=sid)

(100, 100)

In [24]:
c.watermarks("products")

{'products': {0: (100, 2395)}}

In [25]:
c.tail("products", type="bytes", n=1)

[{'topic': 'products',
  'headers': None,
  'partition': 0,
  'offset': 2294,
  'timestamp': (1, 1731606953795),
  'key': None,
  'value': b'\x00\x00\x00\x00\x02{"name": "Chocolate bars", "price": "2.35", "url": "https://www.migros.ch/en/product/100103000000", "id": "100103000000", "version": "Milk chocolate", "price_unit": "0.59/100g", "weight": "400g"}'}]

***

And let's run the qualification test one (hopefully last) time...

(run consumer.py)

***

Very last step: Let's create a copy of that fixed topic in Parquet format for the analytics team - on S3.

In [ ]:
# Connect to S3
s = S3("local")

# Copy the Kafka topic to a Parquet file on S3
c.to_file("products", s, "products.parquet", type="jsonschema")

Read: 1000
Read: 2000


2295

%6|1731621715.759|FAIL|rdkafka#producer-24| [thrd:localhost:9092/bootstrap]: localhost:9092/1: Disconnected while requesting ApiVersion: might be caused by incorrect security.protocol configuration (connecting to a SSL listener?) or broker version is < 0.10 (see api.version.request) (after 1ms in state APIVERSION_QUERY)
%6|1731621715.759|FAIL|rdkafka#producer-18| [thrd:localhost:9092/bootstrap]: localhost:9092/1: Disconnected while requesting ApiVersion: might be caused by incorrect security.protocol configuration (connecting to a SSL listener?) or broker version is < 0.10 (see api.version.request) (after 1ms in state APIVERSION_QUERY)
%6|1731621715.765|FAIL|rdkafka#producer-3| [thrd:localhost:9092/bootstrap]: localhost:9092/1: Disconnected while requesting ApiVersion: might be caused by incorrect security.protocol configuration (connecting to a SSL listener?) or broker version is < 0.10 (see api.version.request) (after 0ms in state APIVERSION_QUERY)
%3|1731621716.008|FAIL|rdkafka#prod

(show Parquet file + look up a product on the Migros web site)

***

That's really it.

Thanks to all my colleagues from Migros in Zürich, in particular the Data Integration team, especially Martin and Jason - the Kafka guys. 

***

Blatant advertising follows.

Get your copy of Kafi from GitHub: https://github.com/xdgrulez/kafi or just install it from PyPI ("kafi")

Get your copy of the new O'Reilly book "Streaming Databases" by Hubert Dulay and me.

![alt text](sdb.jpg "Streaming Databases")


And hand in exciting abstracts to the new, non-vendor-centric conference about everything events and streaming: EventCentric 2025 (Antwerp, Belgium, June 2-5, 2025) https://eventcentric.eu
